In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.models as models
import torch.utils.data as data
import sklearn.utils as utils
from PIL import Image
import torchvision.transforms as T
import sklearn.metrics as metrics

In [ ]:
input_path = '/kaggle/input/cassava-leaf-disease-classification/'

In [ ]:
df = pd.read_csv(os.path.join(input_path, 'train.csv'))

In [ ]:
df

In [ ]:
num_classes = len(df.label.unique())
num_classes

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.base = nn.Sequential(*list(models.resnet101(pretrained=False).children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dense = nn.Linear(2048, num_classes)
    def forward(self, x):
        x = self.base(x)
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        return self.dense(x)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Model().to(device)
# model = nn.DataParallel(model)

In [ ]:
model.load_state_dict(torch.load('/kaggle/input/resnet-cassava-model/model_101_20.pth'))

In [ ]:
submission_df = pd.read_csv(os.path.join(input_path, 'sample_submission.csv'))

In [ ]:
submission_df

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, df, transforms):
        super(TestDataset, self).__init__()
        self.df = df
        self.transforms = transforms
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = Image.open(f'{input_path}/test_images/{row.image_id}')
        return self.transforms(img)
    def __len__(self):
        return len(self.df)

In [ ]:
test_transforms = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5),)
])

In [ ]:
train_dataset = TestDataset(df=submission_df, transforms=test_transforms)
test_dataloader = data.DataLoader(dataset=train_dataset, shuffle=False, batch_size=12)

In [ ]:
model.eval()
test_preds = []
for x in test_dataloader:
    x = x.cuda()
    y_pred = model(x).detach()
    test_preds += y_pred.cpu().numpy().argmax(1).tolist()
test_preds = np.array(test_preds)

In [ ]:
submission_df['label'] = test_preds

In [ ]:
submission_df.to_csv('submission.csv', index=False)